In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
def map_weight(value):
    if (value >= '2022-01') and (value <= '2022-06'):
        return 2
    elif (value >= '2021-07') and (value <= '2021-12'):
        return 1
    elif (value >= '2021-01') and (value <= '2021-06'):
        return 0.5
    else:
        return 1

model_data['Weight_vintage'] = model_data['month_year'].apply(map_weight)


In [4]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2832) (12128, 2832)


In [6]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',

    
#ID Features
'dm206s',
'g406s',


# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob',
    'Weight_vintage'
              
]




In [7]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3161275/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3161275/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 45), (15604, 45), (23406,), (15604,))

In [10]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 45) (4682, 45) (15604, 45)


In [12]:
#Get vintage weight values
train_weights = X_train_new['Weight_vintage'].values
eval_weights = X_eval['Weight_vintage'].values
test_weights = X_test['Weight_vintage'].values
oot_weights = X_oot['Weight_vintage'].values

In [13]:
X_train_new = X_train_new.drop(columns=['Weight_vintage'])
X_eval = X_eval.drop(columns=['Weight_vintage'])
X_test = X_test.drop(columns=['Weight_vintage'])
X_oot = X_oot.drop(columns=['Weight_vintage'])

In [14]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new, weight=train_weights)
deval = xgb.DMatrix(X_eval, label=y_eval,weight=eval_weights)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [15]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,

    'subsample': 0.4,
    'colsample_bytree': 0.4,
     'min_child_weight':25,

    'lambda': 5.5,
    'alpha':5.0,
    'gamma':4.5,

    'scale_pos_weight':3.8,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 600,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61485	eval-auc:0.62275
[10]	train-auc:0.64988	eval-auc:0.65586
[20]	train-auc:0.65449	eval-auc:0.65805
[30]	train-auc:0.65864	eval-auc:0.66102
[40]	train-auc:0.65876	eval-auc:0.66116
[50]	train-auc:0.66049	eval-auc:0.66145
[60]	train-auc:0.66164	eval-auc:0.66114
[70]	train-auc:0.66229	eval-auc:0.66158
[80]	train-auc:0.66396	eval-auc:0.66231
[90]	train-auc:0.66456	eval-auc:0.66286
[100]	train-auc:0.66563	eval-auc:0.66335
[110]	train-auc:0.66668	eval-auc:0.66357
[120]	train-auc:0.66781	eval-auc:0.66394
[130]	train-auc:0.66888	eval-auc:0.66498
[140]	train-auc:0.67048	eval-auc:0.66609
[150]	train-auc:0.67151	eval-auc:0.66674
[160]	train-auc:0.67223	eval-auc:0.66772
[170]	train-auc:0.67290	eval-auc:0.66843
[180]	train-auc:0.67413	eval-auc:0.66899
[190]	train-auc:0.67530	eval-auc:0.66992
[200]	train-auc:0.67613	eval-auc:0.67037
[210]	train-auc:0.67719	eval-auc:0.67020
[220]	train-auc:0.67764	eval-auc:0.67017
[230]	train-auc:0.67858	eval-auc:0.67063
[240]	train-auc:0.67921	eva

In [16]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.27
Eval AUC: 0.67, Gini: 0.34, KS: 0.27
Test AUC: 0.67, Gini: 0.34, KS: 0.25
OOT AUC: 0.68, Gini: 0.36, KS: 0.27


In [17]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.396156         0.647141        0.809465        0.581153   1873   
9          0.254808         0.544680        0.581086        0.512715   1872   
8          0.207265         0.485532        0.512683        0.460089   1872   
7          0.179925         0.438109        0.460088        0.417696   1873   
6          0.171474         0.400120        0.417692        0.383159   1872   
5          0.140491         0.367986        0.383147        0.353904   1872   
4          0.115323         0.341164        0.353901        0.328357   1873   
3          0.102564         0.315353        0.328354        0.302234   1872   
2          0.069979         0.288113        0.302232        0.273165   1872   
1          0.044314         0.244948        0.273072        0.150123   1873   

        label_good  label_bad  
Decile                         
10            1131        742  
9             1395        477  
8             1484        388  
7             1536        337  
6             1551        321  
5             1609        263  
4             1657        216  
3             1680        192  
2             1741        131  
1             1790         83

In [18]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.368354         0.646033        0.816162        0.582052   1561   
9          0.270513         0.543553        0.581943        0.509988   1560   
8          0.216667         0.481822        0.509966        0.457269   1560   
7          0.175529         0.436360        0.457265        0.416337   1561   
6          0.160256         0.398992        0.416336        0.382642   1560   
5          0.142308         0.367596        0.382498        0.353644   1560   
4          0.117873         0.340982        0.353625        0.328287   1561   
3          0.104487         0.315666        0.328282        0.302283   1560   
2          0.091026         0.288211        0.302234        0.272730   1560   
1          0.062140         0.243912        0.272578        0.160959   1561   

        label_good  label_bad  
Decile                         
10             986        575  
9             1138        422  
8             1222        338  
7             1287        274  
6             1310        250  
5             1338        222  
4             1377        184  
3             1397        163  
2             1418        142  
1             1464         97

In [22]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.372630         0.644087        0.843278        0.581324   1213   
9          0.262160         0.542837        0.581318        0.510572   1213   
8          0.224237         0.484536        0.510521        0.461249   1213   
7          0.218647         0.438296        0.461062        0.418001   1212   
6          0.142622         0.399001        0.417984        0.381103   1213   
5          0.147568         0.366730        0.381083        0.352258   1213   
4          0.119637         0.340323        0.352251        0.328688   1212   
3          0.103875         0.315992        0.328625        0.303596   1213   
2          0.074196         0.289628        0.303582        0.275827   1213   
1          0.056884         0.246510        0.275822        0.171298   1213   

        label_good  label_bad  
Decile                         
10             761        452  
9              895        318  
8              941        272  
7              947        265  
6             1040        173  
5             1034        179  
4             1067        145  
3             1087        126  
2             1123         90  
1             1144         69

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
37                                              g406s       162.0
8                highest_sanction_amount_credit_cards       134.0
12                         Age_of_oldest_open_account       122.0
30                                cash_withdrawal_sum        91.0
0                        Num_Enq_all_loans_last_9M_3M        82.0
43                          Num_Enq_non_BANK_last_12M        78.0
1                                     Utilization_all        74.0
4            Utilization_PERSONAL LOAN_opened_last_9M        67.0
38                         percentage_ever_delinquent        60.0
34  deviation_from_median_income/standard_deviatio...        58.0
3             Utilization_CREDIT CARD_opened_last_36M        57.0
42                           No of enq in past 90days        55.0
18                         bounced_transactions_count        50.0
39   months_since_most_recent_delinquency_installment        44.0
9         

In [17]:
feature_importance_df

Feature  Importance
37                                              g406s       162.0
8                highest_sanction_amount_credit_cards       134.0
12                         Age_of_oldest_open_account       122.0
30                                cash_withdrawal_sum        91.0
0                        Num_Enq_all_loans_last_9M_3M        82.0
43                          Num_Enq_non_BANK_last_12M        78.0
1                                     Utilization_all        74.0
4            Utilization_PERSONAL LOAN_opened_last_9M        67.0
38                         percentage_ever_delinquent        60.0
34  deviation_from_median_income/standard_deviatio...        58.0
3             Utilization_CREDIT CARD_opened_last_36M        57.0
42                           No of enq in past 90days        55.0
18                         bounced_transactions_count        50.0
39   months_since_most_recent_delinquency_installment        44.0
9                       num_CONSUMER LOAN_enq_last_6M        44.0
5                         num_CREDIT CARD_enq_last_6M        43.0
14                                    upi_trans_count        42.0
7              Num_of_PERSONAL LOAN_opened_last6M_12M        39.0
22                            days_balance_lt_5000_M3        35.0
21                                     min_balance_M2        34.0
41             Num_of_installment_loan_opened_last_6M        27.0
35                loan_amount_to_income#last_365_Days        26.0
33             ratio_6th_day_balance_to_max_credit_M3        24.0
20                          min_balance_last_3_months        23.0
15           ratio_balance_after_7_days_max_credit_M3        23.0
32                                 balance_6th_day_M2        22.0
6                      num_CONSUMER LOAN_enq_last_36M        22.0
2                                           total_emi        22.0
13                               num_credit_gt_salary        21.0
27                                sum_insurance_debit        21.0
23                           days_balance_lt_10000_M3        19.0
10                 num_PL_LE_1_Lac_opened_in_last_12M        19.0
40                       percentage_utilization_gt_75        18.0
36                                             dm206s        16.0
28                       avg_num_debits_last_2_months        16.0
16                ratio_max_credit_to_total_credit_M2        16.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M        15.0
17                              spend_vs_salary_ratio        14.0
29                                upi_trans_debit_sum        11.0
25                           Max_Credit_Diff_M1_M2_M3         9.0
19              ratio_total_debit_to_max_credit_M2_M3         7.0
24                            Max_Credit_Avg_M1_M2_M3         6.0
26                        amount_debits_last_2_months         6.0
31            diff_btw_stated_salary_vs_actual_salary         6.0

In [21]:
# Save the pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILA_Train_Pred_9024_Vin.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_9024_Vin.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_9024_Vin.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_9024_Vin.csv',index=False)


In [20]:
#Save the model
xgb_model.save_model('LeoPayu_BBILA_weightVintage_90_in_24mob.json')
print('Model Saved')

Model Saved
